In [ ]:
!pip install numpy
!pip install torch
!pip install transformers
!pip install tensorflow
!pip install wandb

In [ ]:
from transformers import RobertaTokenizer
import torch
from pathlib import Path
from tqdm.auto import tqdm
import random
import os
import wandb

In [ ]:
!python3 -m wandb login eb7b1964fb84cd81de96b2a273ecf2bb6254aeac

In [ ]:
radius = 'ecfp0'
folder = 'ecfps_cutted'
folder_with_paths = folder + '/' + radius
samples_count = '2M'
model_name = f'molberto_{radius}_{samples_count}'

In [ ]:
# initialize the tokenizer using the tokenizer we initialized and saved to file
tokenizer = RobertaTokenizer.from_pretrained(model_name, max_len=512)

In [ ]:
def mlm(tensor):
    # create random array of floats with equal dims to tensor
    rand = torch.rand(tensor.shape)
    # mask random 15% where token is not 0 <s>, 1 <pad>, or 2 <s/>
    mask_arr = (rand < .15) * (tensor != 0) * (tensor != 1) * (tensor != 2)
    # loop through each row in tensor (cannot do in parallel)
    for i in range(tensor.shape[0]):
        # get indices of mask positions from mask array
        selection = torch.flatten(mask_arr[i].nonzero()).tolist()
        # mask tensor
        tensor[i, selection] = 4
    return tensor

In [ ]:
paths = [str(x) for x in Path(folder_with_paths).glob('*.txt')]
len(paths)

In [ ]:
# take sentences for training
def tokenize_ecfps(start, end):
    input_ids = []
    mask = []
    labels = []
    for path in tqdm(paths[start:end]):
        with open(path, 'r', encoding='utf-8') as fp:
            lines = fp.read().split('\n')
        sample = tokenizer(lines, max_length=512, truncation=True, padding='max_length')
        labels.append(torch.tensor(sample.input_ids))
        mask.append(torch.tensor(sample.attention_mask))
        input_ids.append(mlm(labels[-1].detach().clone())) # mask ~15% of tokens to create inputs
    
    input_ids = torch.cat(input_ids)
    mask = torch.cat(mask)
    labels = torch.cat(labels)
    return input_ids, mask, labels

In [ ]:
train_input_ids, train_mask, train_labels = tokenize_ecfps(0, 200)
validation_input_ids, validation_mask, validation_labels = tokenize_ecfps(200, 220)
test_input_ids, test_mask, test_labels = tokenize_ecfps(220, 238)

In [ ]:
print(train_input_ids.shape)
print(validation_input_ids.shape)
print(validation_input_ids.shape)

In [ ]:
train_labels[0]

In [ ]:
# torch.save(input_ids, 'molberto_training/input_ids.pt')
# torch.save(mask, 'molberto_training/attention_mask.pt')
# torch.save(labels, 'molberto_training/labels.pt')

# del input_ids, mask, labels

In [ ]:
# input_ids = torch.load('molberto_training/input_ids.pt')
# mask = torch.load('molberto_training/attention_mask.pt')
# labels = torch.load('molberto_training/labels.pt')

### dataset and dataloader

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return self.encodings['input_ids'].shape[0]

    def __getitem__(self, i):
        return {key: tensor[i] for key, tensor in self.encodings.items()}

In [ ]:
train_dataset = Dataset({'input_ids': train_input_ids, 'attention_mask': train_mask, 'labels': train_labels})
validation_dataset = Dataset({'input_ids': validation_input_ids, 'attention_mask': validation_mask, 'labels': validation_labels})
test_dataset = Dataset({'input_ids': test_input_ids, 'attention_mask': test_mask, 'labels': test_labels})

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
validation_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=32, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

And move onto building our model, we first need to create a RoBERTa config object, which will describe which features we want to initialize our RoBERTa model with.

In [ ]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=30_522,  # we align this to the tokenizer vocab set in previous notebook
    max_position_embeddings=514,
    hidden_size=768,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1
)

Then we import and initialize a RoBERTa model with a language modeling head.

In [ ]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config)

And now we move onto training. First we setup GPU/CPU usage.

In [ ]:
torch.cuda.is_available()

In [ ]:
device = torch.device('cuda', index=2) if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
model.to(device)

Activate the training mode of our model, and initialize our optimizer (Adam with weighted decay - reduces chance of overfitting).

In [ ]:

from transformers import AdamW

# activate training mode
model.train()
# initialize optimizer
optim = AdamW(model.parameters(), lr=1e-5)

In [ ]:
wandb.init(
    project="efcp_transformer",
    name="RobertaForMLM on ecpf0 training (2M)",
    config={}
)

Now we move onto the training loop.

In [ ]:
from tqdm import tqdm  # for our progress bar
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

epochs = 1
step = 0

validation_iterator = iter(validation_loader)
for epoch in range(epochs):
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        outputs = model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        loss = outputs.loss
        logits = outputs.logits
        true_labels = batch['labels'].numpy().flatten()
        pred_labels = torch.nn.functional.softmax(logits, dim=1).argmax(axis=-1).cpu().detach().numpy().flatten()

        # write down loss and metrics
        wandb.log({"loss/train": loss}, step=step)
        wandb.log({"accuracy/train": accuracy_score(true_labels, pred_labels)}, step=step)
        wandb.log({"f1/train": f1_score(true_labels, pred_labels, average='micro')}, step=step)
        wandb.log({"precision/train": precision_score(true_labels, pred_labels, average='micro')}, step=step)
        wandb.log({"recall/train": recall_score(true_labels, pred_labels, average='micro')}, step=step)
        
        loss.backward()
        optim.step()
        optim.zero_grad()

        with torch.no_grad():
            try:
                validation_batch = next(validation_iterator)
            except StopIteration:
                validation_dataset = Dataset({'input_ids': validation_input_ids, 'attention_mask': validation_mask, 'labels': validation_labels})
                validation_batch = iter(validation_dataset)
                validation_batch = next(validation_iterator)
            
            input_ids = validation_batch['input_ids'].to(device)
            attention_mask = validation_batch['attention_mask'].to(device)
            labels = validation_batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)

            loss = outputs.loss
            logits = outputs.logits
            true_labels = batch['labels'].numpy().flatten()
            pred_labels = torch.nn.functional.softmax(logits, dim=1).argmax(axis=-1).cpu().detach().numpy().flatten()
    
            # write down loss and metrics
            wandb.log({"loss/validation": loss}, step=step)
            wandb.log({"accuracy/validation": accuracy_score(true_labels, pred_labels)}, step=step)
            wandb.log({"f1/validation": f1_score(true_labels, pred_labels, average='micro')}, step=step)
            wandb.log({"precision/validation": precision_score(true_labels, pred_labels, average='micro')}, step=step)
            wandb.log({"recall/validation": recall_score(true_labels, pred_labels, average='micro')}, step=step)
            
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())
        if step > 2500:
            break
        step += len(batch)


In [ ]:
wandb.finish()

In [ ]:
wandb.init(
    project="efcp_transformer",
    name="RobertaForMLM on ecpf0 testing (2M)",
    config={}
)

In [ ]:
step = 0

with torch.no_grad():
    loop = tqdm(test_loader, leave=True)
    for batch in loop:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        outputs = model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        loss = outputs.loss
        logits = outputs.logits
        true_labels = batch['labels'].numpy().flatten()
        pred_labels = torch.nn.functional.softmax(logits, dim=1).argmax(axis=-1).cpu().detach().numpy().flatten()

        # write down loss and metrics
        wandb.log({"loss/test": loss}, step=step)
        wandb.log({"accuracy/test": accuracy_score(true_labels, pred_labels)}, step=step)
        wandb.log({"f1/test": f1_score(true_labels, pred_labels, average='micro')}, step=step)
        wandb.log({"precision/test": precision_score(true_labels, pred_labels, average='micro')}, step=step)
        wandb.log({"recall/test": recall_score(true_labels, pred_labels, average='micro')}, step=step)
        
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())
        step += len(batch)

wandb.finish()

In [ ]:
wandb.finish()

In [ ]:
model.save_pretrained(f'molberto_{radius}_2.5K_dummy')

In [ ]:
torch.cuda.empty_cache()

In [ ]:
print(torch.cuda.device_count())
print(torch.cuda.current_device())

In [ ]:
torch.device('cuda', index=1)